## Data preprocessing

In [ ]:
import os
import pandas as pd

# import xlsread
  
raw = pd.read_excel(io=r'C:/Users/xiang-group/YNY/CSIS/GW1.xlsx')

l = [] # the index of cells with genes less than 1000
sum0 = (raw == 0).sum(axis=0)
for i in range(2395):
    if sum0[i]>=23153:
        l.append(i)

column = [j for j in raw]
new1 = raw
for i in l:
    new1 = new1.drop(column[i+1],axis=1)

k = [] # the index of genes with express cell less than 3
sum0 = raw.iloc[:,1:].ge(1).sum(axis = 1)
for i in range(24153):
    if sum0[i]<=100:
        k.append(i)

row = raw.index.values
new2 = new1
for i in k:
    new2 = new2.drop(row[i],axis=0)

new2.to_excel('C:/Users/xiang-group/YNY/CSIS/newda.xlsx')
#new2 is the final dataset with the size 19712 × 2345

## Generate the color list

In [ ]:
X = new2.values
co = []
for i in new2.columns.values[1:]:
    co.append( int(i.split("_")[0][2:4]))

## LLE & MLLE

In [ ]:
%matplotlib inline
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.datasets import make_swiss_roll
import matplotlib.pyplot as plt


lle = LocallyLinearEmbedding(method='standard',n_neighbors=12, n_components=2) # or method = 'modified'
X_lle = lle.fit_transform(np.transpose(X[:,1:]))

scatter = plt.scatter(X_lle[:, 0], X_lle[:, 1], c=co, s=5)
plt.xlabel('component1')
plt.ylabel('component2')
plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
plt.xticks([])
plt.yticks([])
plt.legend(*scatter.legend_elements(),
                     title="gestational \n weeks",loc='upper left', bbox_to_anchor=(1.02, 1.0))

plt.savefig('C:/Users/xiang-group/YNY/CSIS/LLE.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## MDS

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components=2)
X_mds = mds.fit_transform(np.transpose(X[:,1:]))

scatter = plt.scatter(X_mds[:, 0], X_mds[:, 1], c=co, s=5)
# plt.set_title('LLE embedded data')
# plt.title('LLE')
plt.xlabel('component1')
plt.ylabel('component2')
plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
plt.xticks([])
plt.yticks([])
plt.xlim(-100000, 200000)
plt.ylim(-120000, 100000)
plt.legend(*scatter.legend_elements(),
                     title="gestational \n weeks",loc='upper left', bbox_to_anchor=(1.02, 1.0))

plt.savefig('C:/Users/xiang-group/YNY/CSIS/MDS.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## t-SNE

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(np.transpose(X[:,1:]))

scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=co, s=5)
# plt.set_title('LLE embedded data')
# plt.title('LLE')
plt.xlabel('component1')
plt.ylabel('component2')
plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
plt.xticks([])
plt.yticks([])
plt.xlim(-40, 50)
plt.ylim(-25, 30)
plt.legend(*scatter.legend_elements(),
                     title="gestational \n weeks",loc='upper left', bbox_to_anchor=(1.02, 1.0))

plt.savefig('C:/Users/xiang-group/YNY/CSIS/tSNE.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## Isomap

In [ ]:
from sklearn.manifold import Isomap

isomap = Isomap(n_components=10)
X_isomap = isomap.fit_transform(np.transpose(X[:,1:]))

scatter = plt.scatter(X_isomap[:, 0], X_isomap[:, 1], c=co, s=5)
plt.xlabel('component1')
plt.ylabel('component2')
plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
plt.xticks([])
plt.yticks([])
plt.xlim(-200000, 150000)
plt.ylim(-150000, 100000)
plt.legend(*scatter.legend_elements(),
                     title="gestational \n weeks",loc='upper left', bbox_to_anchor=(1.02, 1.0))

plt.savefig('C:/Users/xiang-group/YNY/CSIS/isomap.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## SpectralEmbedding

In [ ]:
from sklearn.manifold import SpectralEmbedding

embedding = SpectralEmbedding(n_components=2, random_state=42)
X_transformed = embedding.fit_transform(np.transpose(X[:,1:]))

scatter = plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=co, s=5)
plt.xlabel('component1')
plt.ylabel('component2')
plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
plt.xticks([])
plt.yticks([])
plt.legend(*scatter.legend_elements(),
                     title="gestational \n weeks",loc='upper left', bbox_to_anchor=(1.02, 1.0))

plt.savefig('C:/Users/xiang-group/YNY/CSIS/SE.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## Calculate contour coefficient for clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs
# Calculate contour coefficient for different number of clusters
range_n_clusters = [2, 3, 4, 5, 6]
for n_clusters in range_n_clusters:
    fig, ax = plt.subplots()
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X_tsne)
    silhouette_avg = silhouette_score(X_tsne, cluster_labels)
    sample_silhouette_values = silhouette_samples(X_tsne, cluster_labels)
    y_lower, y_upper = 0, 0
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper += size_cluster_i
        color = plt.cm.get_cmap("Spectral")(float(i) / n_clusters)
        ax.barh(range(y_lower, y_upper), ith_cluster_silhouette_values, height=1.0, edgecolor='none', color=color)
        y_lower += size_cluster_i
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax.set_yticks([])
    ax.set_xlim([-0.1, 1])
    ax.set_xlabel("Silhouette coefficient values")
    ax.set_ylabel("Cluster labels")
    ax.set_title("Cluster silhouette plot of n_clusters = {}".format(n_clusters))
    plt.savefig(f'C:/Users/xiang-group/YNY/CSIS/cluster{n_clusters}.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
    plt.show()

## Kmeans + t-SNE (similar to other data reduction methods)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs

kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(X_tsne)
labels = kmeans.labels_


# Obtain the center point of each cluster and the distance from each data point to the center point
centers = kmeans.cluster_centers_
distances = np.sqrt(((X_tsne - centers[:, np.newaxis])**2).sum(axis=2))

# Eliminate noise points based on threshold
threshold = 120
is_noise = distances > threshold
cleaned_X_tsne = X_tsne[~np.any(is_noise, axis=0)]

label_new = labels[~np.any(is_noise, axis=0)]
co_new = np.array(co)[~np.any(is_noise, axis=0)]

fig, ax = plt.subplots()
scatter = ax.scatter(cleaned_X_tsne[:, 0], cleaned_X_tsne[:, 1], c=label_new, cmap='viridis',s=5)


plt.legend(*scatter.legend_elements(),
                     title="categories \n number",loc='upper left', bbox_to_anchor=(1.02, 1.0))
plt.xlabel('Component1')
plt.ylabel('Component2')
plt.xticks([])
plt.yticks([])
plt.title('K-Means Clustering for data processed by Standard tSNE')
plt.savefig('C:/Users/xiang-group/YNY/CSIS/KM+tsne.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## Spectral clustering + t-SNE

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import SpectralClustering


sc = SpectralClustering(n_clusters=6, affinity='nearest_neighbors', assign_labels='kmeans')
sc.fit(X_tsne)
labels = sc.labels_


# Obtain the center point of each cluster and the distance from each data point to the center point
# Compute the cluster centers
centers = []
for i in range(sc.n_clusters):
    cluster_points = X_tsne[labels == i]
    center = np.mean(cluster_points, axis=0)
    centers.append(center)
centers = np.array(centers)
distances = np.sqrt(((X_tsne - centers[:, np.newaxis])**2).sum(axis=2))

# Eliminate noise points based on threshold
threshold = 86
is_noise = distances > threshold
cleaned_X_tsne = X_tsne[~np.any(is_noise, axis=0)]

label_new = labels[~np.any(is_noise, axis=0)]
co_new = np.array(co)[~np.any(is_noise, axis=0)]

fig, ax = plt.subplots()
scatter = ax.scatter(cleaned_X_tsne[:, 0], cleaned_X_tsne[:, 1], c=label_new, cmap='viridis',s=5)


plt.legend(*scatter.legend_elements(),
                     title="categories \n number",loc='upper left', bbox_to_anchor=(1.02, 1.0))
plt.xlabel('Component1')
plt.ylabel('Component2')
plt.xticks([])
plt.yticks([])
plt.title('Spectral Clustering for data processed by tSNE')
plt.savefig('C:/Users/xiang-group/YNY/CSIS/SC+tsne.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## (ward) Hierarchical clustering + t-SNE

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering


hc = AgglomerativeClustering(n_clusters=6)
hc.fit(X_tsne)
labels = hc.labels_


# Obtain the center point of each cluster and the distance from each data point to the center point
# Compute the cluster centers
centers = []
for i in range(hc.n_clusters):
    cluster_points = X_tsne[labels == i]
    center = np.mean(cluster_points, axis=0)
    centers.append(center)
centers = np.array(centers)
distances = np.sqrt(((X_tsne - centers[:, np.newaxis])**2).sum(axis=2))

# Eliminate noise points based on threshold
threshold = 90
is_noise = distances > threshold
cleaned_X_tsne = X_tsne[~np.any(is_noise, axis=0)]

label_new = labels[~np.any(is_noise, axis=0)]
co_new = np.array(co)[~np.any(is_noise, axis=0)]

fig, ax = plt.subplots()
scatter = ax.scatter(cleaned_X_tsne[:, 0], cleaned_X_tsne[:, 1], c=label_new, cmap='viridis',s=5)


plt.legend(*scatter.legend_elements(),
                     title="categories \n number",loc='upper left', bbox_to_anchor=(1.02, 1.0))
plt.xlabel('Component1')
plt.ylabel('Component2')
plt.xticks([])
plt.yticks([])
plt.title('Hierarchical Clustering for data processed by tSNE')
plt.savefig('C:/Users/xiang-group/YNY/CSIS/HC+tsne.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()

## BIRCH clustering + t-SNE

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import Birch


birch = Birch(threshold=0.1, n_clusters=6)
birch.fit(X_tsne)
labels = birch.labels_



# Obtain the center point of each cluster and the distance from each data point to the center point
# Compute the cluster centers
centers = birch.subcluster_centers_
distances = np.sqrt(((X_tsne - centers[:, np.newaxis])**2).sum(axis=2))

# Get the number of clusters
n_clusters = len(centers)

# Eliminate noise points based on threshold
threshold = 103.5
is_noise = distances > threshold
cleaned_X_tsne = X_tsne[~np.any(is_noise, axis=0)]

label_new = labels[~np.any(is_noise, axis=0)]
co_new = np.array(co)[~np.any(is_noise, axis=0)]

fig, ax = plt.subplots()
scatter = ax.scatter(cleaned_X_tsne[:, 0], cleaned_X_tsne[:, 1], c=label_new, cmap='viridis',s=5)


plt.legend(*scatter.legend_elements(),
                     title="categories \n number",loc='upper left', bbox_to_anchor=(1.02, 1.0))
plt.xlabel('Component1')
plt.ylabel('Component2')
plt.xticks([])
plt.yticks([])
plt.title('BIRCH Clustering for data processed by tSNE')
plt.savefig('C:/Users/xiang-group/YNY/CSIS/birch+tsne.png',dpi=300, bbox_inches='tight',facecolor='white', edgecolor='white')
plt.show()